In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xarray as xr
import os

import hydrofunctions as hf
%matplotlib inline

In [ ]:
start_date = "2021-11-10"
end_date = "2021-12-10"
date_list = pd.date_range(start_date, end_date).to_list()

In [ ]:
precip = xr.open_dataset(f'climate_data/ppt_{start_date}-{end_date}.nc')
tmean = xr.open_dataset(f'climate_data/tmean_{start_date}-{end_date}.nc')
snow_accumulation = xr.open_dataset(f'climate_data/snow_{start_date}-{end_date}.nc')
snow_change = xr.open_dataset(f'climate_data/snow_change_{start_date}-{end_date}.nc')

In [ ]:
total_runoff = []
for i in range(len(date_list)):
    runoff = precip['ppt'].isel(date=i) - snow_change['__xarray_dataarray_variable__'].isel(date=i)
    runoff.coords['date'] = date_list[i]
    total_runoff.append(runoff)

total_runoff = xr.concat(total_runoff, 'date')

In [ ]:
total_runoff_volume_gridded = total_runoff * (10**-3) * (4000*4000)
total_runoff_volume_gridded.isel(date=slice(0, 31, 1)).plot.imshow(col = 'date', col_wrap = 3, vmin = 0)

In [ ]:
daily_runoff_volume = total_runoff_volume_gridded.sum(dim = ('lat', 'lon'))

In [ ]:
daily_runoff_volume

In [ ]:
discharge_df = pd.read_parquet('discharge_data/everson.parquet', engine = 'pyarrow')

In [ ]:
discharge_df.head()

In [ ]:
daily_discharge_volume = discharge_df['USGS:12211200:00060:00003']*60*60*24/35.314666212661

In [ ]:
daily_streamflow_difference = []
for i in range(len(date_list)):
    daily_delta = daily_runoff_volume.isel(date=i).values - daily_discharge_volume.iloc[i]
    daily_streamflow_difference.append(daily_delta)

In [ ]:
daily_streamflow_difference

In [ ]:
f, a = plt.subplots()
a.plot(date_list, daily_streamflow_difference)
a.axhline(0, color = 'k')